In [31]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
import os
import re
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from joblib import dump, load
import ltn

dataset_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU'

PGB_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU\\PGB'
RGB_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU\\RGB'

# Specify the CSV file path
csv_file = 'C:\\work_ssd\\DDS_Paper\\data\\data_robust.csv'
preprocessor_file = 'preprocessor.joblib'


np.random.seed(45)

# Set the chunk size for reading the CSV
chunk_size = 100000  # Adjust the chunk size according to your memory limitations

In [32]:
def extract_fault(file_name):
    match = re.search(r'\d+', file_name)
    if match:
        return int(match.group(0)[0])  # Extract the first digit
    else:
        return None

def process_files_to_csv(data_folders, output_file):
    # Check if the file already exists
    if os.path.isfile(output_file):
        print(f"File {output_file} already exists. Skipping processing.")
        return

    total_files = sum([len(files) for data_folder in data_folders for r, d, files in os.walk(data_folder)])

    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Speed', 'Type', 'Fault'])

        with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
            for data_folder in data_folders:
                for root, dirs, files in os.walk(data_folder):
                    if '.ipynb_checkpoints' in root:
                        continue  # Skip .ipynb_checkpoints folders
                    for file in files:
                        if file.endswith('.txt'):
                            file_path = os.path.join(root, file)
                            path_parts = file_path.split('\\')
                            variable_speed = 'Variable_speed' in file_path
                            type_index = -4 if variable_speed else -3
                            type = path_parts[type_index] if path_parts[type_index] in ['PGB', 'RGB'] else None
                            if type is not None:
                                speed_index = -3 if variable_speed else -2
                                speed = path_parts[speed_index]
                                fault = extract_fault(file)

                                data = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')
                                reshaped_data = data.values[:, :]

                                for row_data in tqdm(reshaped_data, desc="Processing rows", unit="row", leave=False):
                                    row = row_data.tolist() + [speed, type, fault]
                                    csv_writer.writerow(row)
                            pbar.update()

In [33]:
# Example usage
process_files_to_csv([PGB_path, RGB_path], csv_file)

File C:\work_ssd\DDS_Paper\data\data_robust.csv already exists. Skipping processing.


# Data preprocessing

In [4]:

# Initialize a dictionary to store the fault counts
fault_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        
        #print(chunk)
        # Assuming there is a column named 'fault' in the CSV representing the fault type
        fault_column = 'Fault'

        # Count the occurrences of each fault in the current chunk
        fault_chunk_counts = chunk[fault_column].value_counts()

        # Aggregate the counts with the overall fault_counts dictionary
        for fault, count in fault_chunk_counts.items():
            fault_counts[fault] = fault_counts.get(fault, 0) + count

        pbar.update()

# Print the fault counts
for fault, count in fault_counts.items():
    print(f"Fault: {fault}, Count: {count}")


Processing CSV: 7267chunk [09:46, 12.39chunk/s]         

Fault: 0, Count: 80740352
Fault: 1, Count: 80740352
Fault: 2, Count: 80740352
Fault: 3, Count: 80740352
Fault: 4, Count: 80740352
Fault: 5, Count: 80740352
Fault: 6, Count: 80740352
Fault: 7, Count: 80740352
Fault: 8, Count: 80740352


In [5]:
# Initialize dictionaries to store the counts
speed_counts = {}
type_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        # Assuming there is a column named 'Speed' in the CSV representing the speed values
        
        speed_column = 'Speed'

        # Count the occurrences of each speed value in the current chunk
        speed_chunk_counts = chunk[speed_column].value_counts()

        # Aggregate the counts with the overall speed_counts dictionary
        for speed, count in speed_chunk_counts.items():
            speed_counts[speed] = speed_counts.get(speed, 0) + count

        # Assuming there is a column named 'Type' in the CSV representing the types
        type_column = 'Type'

        # Count the occurrences of each type in the current chunk
        type_chunk_counts = chunk[type_column].value_counts()

        # Aggregate the counts with the overall type_counts dictionary
        for typ, count in type_chunk_counts.items():
            type_counts[typ] = type_counts.get(typ, 0) + count

        pbar.update()

# Print the speed counts
print("Speed Counts:")
for speed, count in speed_counts.items():
    print(f"Speed: {speed}, Count: {count}")

# Print the type counts
print("Type Counts:")
for typ, count in type_counts.items():
    print(f"Type: {typ}, Count: {count}")


Processing CSV: 7267chunk [10:25, 11.62chunk/s]         

Speed Counts:
Speed: 20_0, Count: 75497472
Speed: 30_0, Count: 75497472
Speed: 30_1, Count: 75497472
Speed: 30_2, Count: 75497472
Speed: 30_3, Count: 75497472
Speed: 30_4, Count: 75497472
Speed: 30_5, Count: 75497472
Speed: 40_0, Count: 75497472
Speed: 50_0, Count: 75497472
Speed: Variable_speed, Count: 47185920
Type Counts:
Type: PGB, Count: 363331584
Type: RGB, Count: 363331584


In [34]:
# specify the numeric and categorical features
numerical_features = ['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8']
categorical_features = ['Speed', 'Type']

# initialize the scalers
scaler = StandardScaler()
encoder = OneHotEncoder(categories=[['20_0','30_0','30_1','30_2', '30_3','30_4','30_5','40_0','50_0', 'Variable_speed'], ['PGB', 'RGB']])  # specify the unique values

In [35]:
# define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_features),
        ('cat', encoder, categorical_features)])


In [36]:
# get the total size of the file in bytes
total_size = os.path.getsize(csv_file)

# define the number of lines to read for the estimate
sample_lines = 1000

# read the first few lines of the file to get the average bytes per line
with open(csv_file, 'r') as file:
    lines = [next(file) for _ in range(sample_lines)]
avg_bytes_per_line = sum(len(line) for line in lines) / sample_lines

# estimate the total number of lines in the file
estimated_lines = total_size // avg_bytes_per_line

# calculate the estimated total number of chunks
total_chunks = estimated_lines // chunk_size
if estimated_lines % chunk_size != 0:
    total_chunks += 1
    
# generate a boolean mask for the entire dataset
mask = np.random.rand(int(estimated_lines)) < 0.8


In [37]:
if not os.path.exists(preprocessor_file):
    # initialize a progress bar
    progress_bar = tqdm(total=total_chunks, desc="Fitting Scaler and Encoder", unit="chunk")

    # fit the scaler and encoder incrementally
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        chunk_mask = np.random.rand(len(chunk)) < 0.8
        train_chunk = chunk[chunk_mask]
        scaler.partial_fit(train_chunk[numerical_features])
        encoder.fit(train_chunk[categorical_features])
        progress_bar.update()

    # close the progress bar
    progress_bar.close()
else:
    print("Preprocessor file already exists.")

Preprocessor file already exists.


In [38]:
if not os.path.exists(preprocessor_file):
    # Save the preprocessor
    dump(preprocessor, 'preprocessor.joblib')

    # Load the preprocessor when needed
    preprocessor = load('preprocessor.joblib')
else:
    print("Preprocessor file already exists.")

Preprocessor file already exists.


In [39]:
if os.path.exists(preprocessor_file):
    progress_bar = tqdm(total=total_chunks, desc="Fitting preprocessor", unit="chunk")

    # fit the preprocessor on the entire training data
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        chunk_mask = np.random.rand(len(chunk)) < 0.8
        train_chunk = chunk[chunk_mask]
        preprocessor.fit(train_chunk)
        progress_bar.update()
    # close the progress bar
    progress_bar.close()
else:
    print("There is no preprocessor file saved.")

Fitting preprocessor:  98%|█████████▊| 7267/7398.0 [15:14<00:16,  7.95chunk/s]


In [40]:
def train_generator():
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        train_chunk = chunk.sample(frac=0.8)  # Select a random 80% fraction of the chunk
        # perform the transformations
        transformed_chunk = preprocessor.transform(train_chunk)
        labels = train_chunk['Fault']  # Assuming 'Fault' is your label column
        yield transformed_chunk, labels

def val_generator():
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        val_chunk = chunk.drop(train_chunk.index)  # Use the rest 20% for validation
        # perform the transformations
        transformed_chunk = preprocessor.transform(val_chunk)
        labels = val_chunk['Fault']  # Assuming 'Fault' is your label column
        yield transformed_chunk, labels

In [41]:
batch_size = 64

# Create tf.data.Datasets for training and validation
train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).batch(batch_size)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 20), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).batch(batch_size)

In [42]:
# train the model on the training dataset
last_train_batch = None
for train_batch in train_dataset.batch(2):
    # perform a training step here
    print(train_batch)
    last_train_batch = train_batch
    break

if last_train_batch is not None:
    print(last_train_batch)

(<tf.Tensor: shape=(2, 64, 80000, 20), dtype=float32, numpy=
array([[[[-3.9093413e+00, -7.2891164e-01,  2.5583293e+00, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         [ 4.7791898e-01,  3.0517551e-01,  4.6552780e-01, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         [ 5.1489401e-01,  1.2404720e+00, -2.4753553e-01, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         ...,
         [-3.7117445e-01, -8.7446713e-01,  1.3548978e-01, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         [-2.5569210e+00,  6.9624639e-01, -1.2177547e+00, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         [-1.7036083e+00, -2.4002437e-01,  1.6545904e+00, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00]],

        [[-2.1897244e+00, -1.8590616e+00, -7.2698200e-01, ...,
           0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
         [-4.0798926e+00, -5.6640387e-01,  4.2033968e-03, ...,
      

## LSTM as backbone

In [43]:
class LSTMModel(tf.keras.Model):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = tf.keras.layers.LSTM(64, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)  # Define dropout layer
        self.lstm2 = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(9)  # no activation function here

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dropout(x, training=training)
        x = self.lstm2(x)
        x = self.dropout(x, training=training)
        return self.dense(x)


# Create an instance of the model
model = LSTMModel()

# Define a predicate using the model
p = ltn.Predicate.FromLogits(model, activation_function="softmax", with_class_indexing=True)

#p = ltn.Predicate.FromLogits(model, activation_function="softmax")


In [44]:
# Constants to index/iterate on the classes
HEALTHY = ltn.Constant(0, trainable=False)
CTF = ltn.Constant(1, trainable=False)
MTF = ltn.Constant(2, trainable=False)
RCF = ltn.Constant(3, trainable=False)
SWF = ltn.Constant(4, trainable=False)
BWF = ltn.Constant(5, trainable=False)
CWF = ltn.Constant(6, trainable=False)
IRF = ltn.Constant(7, trainable=False)
ORF = ltn.Constant(8, trainable=False)

In [24]:
# # Class Variables
# class_healthy = ltn.Variable("class_healthy", HEALTHY)
# class_CTF = ltn.Variable("class_CTF", CTF)
# class_MTF = ltn.Variable("class_MTF", MTF)
# class_RCF = ltn.Variable("class_RCF", RCF)
# class_SWF = ltn.Variable("class_SWF", SWF)
# class_BWF = ltn.Variable("class_BWF", BWF)
# class_CWF = ltn.Variable("class_CWF", CWF)
# class_IRF = ltn.Variable("class_IRF", IRF)
# class_ORF = ltn.Variable("class_ORF", ORF)

In [45]:
#operators
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

Mutual Exclusivity

- Each fault state is mutually exclusive with the other states, meaning that a gearbox can't simultaneously have multiple fault types. You can express this using the logical Not and And operations in LTN.


Single Fault State: 

- Each gearbox has at least one fault state (which could be the healthy state, represented by fault type 0). You could express this using the logical Or operation.


Correlation Between Features and Fault Types .... maybe?

- If you have knowledge about which features are more likely to be affected by specific faults, you can encode this knowledge into the axioms. For instance, if you know that a particular vibration pattern in Channel 1 is indicative of a 'Missing Tooth Fault' (MTF), then whenever that pattern appears, the MTF state should be more likely. Note that this would require a more complex setup, where you divide your features into groups and apply the predicate to these groups.

In [46]:
# Define the formula aggregator
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_healthy = ltn.Variable("x_healthy", features[labels == HEALTHY])
    x_CTF = ltn.Variable("x_CTF", features[labels == CTF])
    x_MTF = ltn.Variable("x_MTF", features[labels == MTF])
    x_RCF = ltn.Variable("x_RCF", features[labels == RCF])
    x_SWF = ltn.Variable("x_SWF", features[labels == SWF])
    x_BWF = ltn.Variable("x_BWF", features[labels == BWF])
    x_CWF = ltn.Variable("x_CWF", features[labels == CWF])
    x_IRF = ltn.Variable("x_IRF", features[labels == IRF])
    x_ORF = ltn.Variable("x_ORF", features[labels == ORF])

    all_faults = [x_healthy, x_CTF, x_MTF, x_RCF, x_SWF, x_BWF, x_CWF, x_IRF, x_ORF]
    all_classes = [class_healthy, class_CTF, class_MTF, class_RCF, class_SWF, class_BWF, class_CWF, class_IRF, class_ORF]

    axioms = [
        Forall(x, p([x, class_], training=training)) for x, class_ in zip(all_faults, all_classes)
    ]

    # Mutual exclusivity
    for i in range(len(all_faults)):
        for j in range(i+1, len(all_faults)):
            axioms.append(Not(And(p([all_faults[i], all_classes[j]], training=training), p([all_faults[j], all_classes[i]], training=training))))

    # Single fault state
    axioms.append(Or([p([x, class_], training=training) for x, class_ in zip(all_faults, all_classes)]))

    sat_level = formula_aggregator(axioms).tensor
    return sat_level


In [47]:
# Function to compute and print the initial satisfaction level
def print_sat_level(data):
    features, labels = data
    print(labels)
    print(HEALTHY)
    labels = tf.squeeze(labels) # ensure labels is a 1-D array
    print("Initial sat level %.5f"%axioms(features,labels))

# Get the first batch of the test dataset
features, labels = next(iter(train_dataset))

# Ensure labels is a 1-D array
labels = tf.squeeze(labels)

# Print the initial satisfaction level for the first batch
print_sat_level((features, labels))


tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(64, 80000), dtype=int32)
ltn.Constant(tensor=[0.], trainable=False, free_vars=[])


TypeError: in user code:

    File "C:\Users\edy45\AppData\Local\Temp\ipykernel_22364\2999002591.py", line 6, in axioms  *
        x_healthy = ltn.Variable("x_healthy", features[labels == HEALTHY])

    TypeError: Expected int32 passed to parameter 'y' of op 'Equal', got ltn.Constant(tensor=[0.], trainable=False, free_vars=[]) of type 'Constant' instead. Error: Expected int32, but got ltn.Constant(tensor=[0.], trainable=False, free_vars=[]) of type 'Constant'.


In [48]:
# Print the initial satisfaction level for each batch of the test dataset
for data in train_dataset:
    print_sat_level(data)

tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(64, 80000), dtype=int32)
ltn.Constant(tensor=[0.], trainable=False, free_vars=[])


TypeError: in user code:

    File "C:\Users\edy45\AppData\Local\Temp\ipykernel_22364\2999002591.py", line 6, in axioms  *
        x_healthy = ltn.Variable("x_healthy", features[labels == HEALTHY])

    TypeError: Expected int32 passed to parameter 'y' of op 'Equal', got ltn.Constant(tensor=[0.], trainable=False, free_vars=[]) of type 'Constant' instead. Error: Expected int32, but got ltn.Constant(tensor=[0.], trainable=False, free_vars=[]) of type 'Constant'.


In [ ]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

In [ ]:
# Adam optimizer for updating model parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Use the built-in tf.data.Dataset API to shuffle and batch the data
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
# Define train_step
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1. - sat
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # compute sat without dropout
    sat = axioms(features, labels, training=False)  # Explicitly specify training mode
    metrics_dict['train_sat_kb'](sat)
    
    # accuracy
    predictions = model(features, training=False)  # get predictions from the model
    metrics_dict['train_accuracy'](tf.one_hot(labels, 9), tf.nn.softmax(predictions))


# Define test_step
@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    
    # accuracy
    predictions = model(features, training=False)  # get predictions from the model
    metrics_dict['test_accuracy'](tf.one_hot(labels, 9), tf.nn.softmax(predictions))



Note: LSTMs may not be the best approach as a backbone. While LSTM models are excellent for sequence prediction problems where temporal dependencies matter However,  the axioms describe the relationships between the predicates and the variables, and do not involve any consideration of time or sequence.

## Training the model

In [ ]:
# Define optimizer, number of epochs, and metrics_dict as before
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
epochs = 500

# Prepare CSV file for saving results
with open("results.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=list(metrics_dict.keys()))
    writer.writeheader()

# Loop over epochs
for epoch in range(epochs):
    # Loop over batches in train_dataset
    for features, labels in train_dataset:
        train_step(features, labels)
    
    # Loop over batches in test_dataset
    for features, labels in val_dataset:
        test_step(features, labels)
    
    # Prepare metrics for saving
    metrics_results = {name: metric.result().numpy() for name, metric in metrics_dict.items()}
    
    # Save metrics to CSV
    with open("results.csv", "a") as f:
        writer = csv.DictWriter(f, fieldnames=list(metrics_dict.keys()))
        writer.writerow(metrics_results)
    
    # Print metrics
    print(f"Epoch {epoch+1}/{epochs}")
    for name, value in metrics_results.items():
        print(f"{name}: {value}")
    
    # Reset metrics at the end of each epoch
    for metric in metrics_dict.values():
        metric.reset_states()